<a href="https://colab.research.google.com/github/ssubbinn/ESAA-OB/blob/main/11%EC%A3%BC%EC%B0%A8_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 파이썬 추천 시스템 패키지-Surprise

파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 : surprise. 장점은 다음과 같다.

- 다양한 추천 알고리즘 (사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링)을 쉽게 적용해 추천 시스템 구축
- fit, predict(), train_test_split, cross_validate, gridsearchcv 등의 클래스를 통해 추천 시스템을 위한 모델 셀렉션 평가, 하이퍼 파라미터 튜닝 등의 기능 제공.

In [1]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095463 sha256=cc8a70097541f0ddacef14cdbd2d7e19dc54b242ceb9f3d91e555776d836535b
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


**Surprise를 이용한 추천 시스템 구축**

추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤, SVD 행렬 분해를 통해 잠재 요인 협업 필터링을 수행.

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
data=Dataset.load_builtin('ml-100k')
#수행시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset=train_test_split(data,test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [10]:
!wget -c https://github.com/euphoris/datasets/raw/master/ml-100k.zip

--2023-05-19 06:21:40--  https://github.com/euphoris/datasets/raw/master/ml-100k.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/euphoris/datasets/master/ml-100k.zip [following]
--2023-05-19 06:21:41--  https://raw.githubusercontent.com/euphoris/datasets/master/ml-100k.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 873734 (853K) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>] 853.26K  --.-KB/s    in 0.05s   

2023-05-19 06:21:41 (15.4 MB/s) - ‘ml-100k.zip’ saved [873734/873734]



In [11]:
from surprise import Reader
import pandas as pd

In [14]:
ratings=pd.read_csv('ml-100k.zip')

In [15]:
ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [4]:
algo=SVD()
algo.fit(trainset) 

- 학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을 수행
- test()(사용자-아이템 평점 데이터 세트 전체에 대해 추천을 예측)와 predict()(개별 사용자와 영화에 대한 추천 평점 반환)이용

In [5]:
#test()의 작동
predictions=algo.test(testset)
print('predictions type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

predictions type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5603575724875096, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.6723948607479224, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9266113683402706, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.572773069231113, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.602772386367124, details={'was_impossible': False})]

In [6]:
#userid, itemid, est에 접근해보자
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5603575724875096),
 ('882', '291', 3.6723948607479224),
 ('535', '507', 3.9266113683402706)]

In [7]:
#predict()의 작동

#사용자 아이디, 아이템 아이디는 문자열로 입력해야함.
uid=str(196)
iid=str(302)
pred=algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.27   {'was_impossible': False}


predict는 이렇게 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 반환.

테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이도 평가해보자.

In [8]:
#바로 rmse 사용
accuracy.rmse(predictions)

RMSE: 0.9490


0.9490098912091891

**Surprise 주요 모듈 소개**

Dataset

로우 레벨이며 네번째 칼럼부터는 아예 로딩을 수행하지 않는 surprise. 따라서 데이터 세트의 칼럼 순서가 반드시 사용자 아이디, 아이템 아이디, 평점 순으로 되어 있어야 한다.

반드시 데이터 세트의 칼럼 순서가 사용자 아이디, 아이템 아이디, 평점 순
Dataset.load_builtin(name='ml-100k')
무비렌즈 아카이브 FTP서버에서 무비 렌즈 데이터를 내려 받는다.

Dataset.load_from_file(file_path, reader)
OS파일에서 데이터를 로딩할 때 사용

Dataset.load_from_df
판다스의 DataFrame에서 데이터를 로디

In [17]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [16]:
import pandas as pd

#ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('ratings_noh.csv', index=False, header=False)

In [18]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/ratings_noh.csv',reader=reader)

In [19]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.9458


0.9457855197571977

In [22]:
import pandas as pd
from surprise import Reader, Dataset

ratings=pd.read_csv('ml-100k.zip')
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.9458


0.9457855197571977


**교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝**

In [23]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings=pd.read_csv('ml-100k.zip') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9329  0.9366  0.9290  0.9312  0.9338  0.9327  0.0026  
MAE (testset)     0.7348  0.7393  0.7321  0.7340  0.7353  0.7351  0.0024  
Fit time          1.34    1.34    1.40    1.36    1.35    1.36    0.02    
Test time         0.28    0.13    0.13    0.13    0.12    0.16    0.06    


{'test_rmse': array([0.93294675, 0.93662588, 0.92895178, 0.93116049, 0.93382461]),
 'test_mae': array([0.73476855, 0.73929516, 0.73209041, 0.73398703, 0.7352941 ]),
 'fit_time': (1.3406271934509277,
  1.343848705291748,
  1.398329734802246,
  1.3612263202667236,
  1.3535501956939697),
 'test_time': (0.28139328956604004,
  0.13078784942626953,
  0.1334524154663086,
  0.13064789772033691,
  0.12315726280212402)}

In [24]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.9431350087827717
{'n_epochs': 20, 'n_factors': 50}



**Surprise 를 이용한 개인화 영화 추천 시스템 구축**

In [25]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

In [26]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='/content/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [27]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [ ]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['user_id']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

In [ ]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

In [ ]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

In [ ]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])